<a href="https://colab.research.google.com/github/AraiKensuke/GCoh/blob/master/simulations/simulated_multichannel_EEG_Gcoh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
#########  clone and install necessary packages from github repositories

import sys
import importlib

if importlib.util.find_spec("GCoh") is None:
  !git clone https://github.com/AraiKensuke/GCoh.git

if importlib.util.find_spec("mne") is None:
  !pip install mne

sys.path.insert(1, "/content/GCoh")    #  add these to searchpath for python modules

In [12]:
import sklearn

In [3]:
#####  Generate simulated EEG data.

import numpy as _N
import os

import GCoh.datconfig as datconf
import GCoh.simulations.sim_utils as _su

#  grpA and grpB   can coexist
#  grpA and grpC   can coexist
#  grpB and grpD   can coexist

grpA = _N.array([1, 2, 5])
grpB = _N.array([3, 4, 8])
grpC = _N.array([7, 8, 9])
grpD = _N.array([1, 6, 7])
grpA_phase_diff = _N.array([0, -2, 5])
grpB_phase_diff = _N.array([0, 13, 9])
grpC_phase_diff = _N.array([0, 1, 1])
grpD_phase_diff = _N.array([0, 0, 0])

dt   = 0.005
nChs = 10
N    = 10000   #(50 s)
f    = 20      #  oscillation frequency 
amp  = 0.995

#  up to 4 groups of
rhythms = _N.zeros((nChs, N))
pknzs   = _N.zeros((nChs, N))
tau     = 0.15    #  timescale of AR1

######  we generate simulated EEG by generating nChs of independent oscillatory
######  signals.  
for nc in range(nChs):
    rhythms[nc] = _su.AR2(f, 0.98, N, dt)
    pknzs[nc] = _su.AR1(tau, N, dt)
EEG     = _N.array(rhythms)

#mix(dt, EEG, rhythms, pknzs, grpA, 10, 15)
_su.mix(dt, EEG, rhythms, pknzs, grpB, grpB_phase_diff, 18, 45)
#mix(dt, EEG, rhythms, pknzs, grpD, 18, 45)
#mix(dt, EEG, rhythms, pknzs, grpC, 28, 34)
#mix(dt, EEG, rhythms, pknzs, grpD, 40, 45)






In [8]:
import numpy as _N
import scipy.signal as _ssig
import mne.time_frequency as mtf
import mne
import pickle
import sys

import GCoh.chronux_py.chronux_funcs as _chrf
import preprocess_ver

import GCoh.datconfig as datconf
import GCoh.windowed_gcoh as _w_gcoh

dataset =    datconf._SIM
Fs      = 200

artrmv_ver = 1
gcoh_ver    = 1

wnd, slideby      = preprocess_ver.get_win_slideby(gcoh_ver)

rm_chs = []
ch_w_CM = _N.arange(10)

X_cm    = EEG.T

ch_names = ["eeg1", "eeg2", "eeg3", "eeg4", "eeg5", "eeg6", "eeg7", "eeg8", "eeg9", "eeg10"]
ch_types = ["eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg", "eeg"]
ch_picks = _N.setdiff1d(ch_w_CM, _N.array(rm_chs))
arr_ch_picks = _N.array(ch_picks)

info = mne.create_info(ch_names=(_N.array(ch_names)[ch_picks]).tolist(), ch_types=(_N.array(ch_types)[ch_picks]).tolist(), sfreq=Fs)

datconf.set_montage(dataset, info)

dpss_bw=7
f, findx, Ctot, Cvec = _w_gcoh.windowed_gcoh(Fs, wnd, slideby, X_cm, ch_w_CM, ch_picks, info, dpss_bw=dpss_bw)

#pkldat = {"VEC" : Cvec, "Cs" : Ctot, "fs" : f[findx], "chs_picks" : arr_ch_picks, "dpss_bw" : dpss_bw}



    Using multitaper spectrum estimation with 8 DPSS windows
Not setting metadata
Not setting metadata
1 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
0  256
128  384
256  512
384  640
512  768
640  896
768  1024
896  1152
1024  1280
1152  1408
1280  1536
1408  1664
1536  1792


/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1

1664  1920
1792  2048
1920  2176
2048  2304
2176  2432
2304  2560
2432  2688
2560  2816
2688  2944
2816  3072
2944  3200
3072  3328
3200  3456
3328  3584
3456  3712


/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1

3584  3840
3712  3968
3840  4096
3968  4224
4096  4352
4224  4480
4352  4608
4480  4736
4608  4864
4736  4992
4864  5120
4992  5248
5120  5376
5248  5504
5376  5632


...

/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1

7552  7808
7680  7936
7808  8064
7936  8192
8064  8320
8192  8448
8320  8576
8448  8704
8576  8832
8704  8960
8832  9088
8960  9216
9088  9344
9216  9472
9344  9600


/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1

9472  9728
9600  9856
9728  9984


/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1, verbose=False)
/content/GCoh/windowed_gcoh.py:66: RuntimeWarning: Epochs are not baseline corrected or enough highpass filtered. Cross-spectral density may be inaccurate.
  csd = mtf.csd_multitaper(epochs, tmin=(t0/Fs), tmax=(t1/Fs), fmin=fMin, fmax=fMax, n_fft=wnd, bandwidth=dpss_bw, adaptive=False, low_bias=True, projs=None, n_jobs=1

In [ ]:
import numpy as _N
import scipy.io as _scio
import scipy.stats as _ss
import matplotlib.pyplot as _plt
from scipy.signal import savgol_filter
from sklearn import mixture
from GCoh.eeg_util import unique_in_order_of_appearance, increasing_labels_mapping, rmpd_lab_trnsfrm, find_or_retrieve_GMM_labels, shift_correlated_shuffle, mtfftc
import GCoh.skull_plot as _sp
import os
import AIiRPS.rpsms as rpsms
import GCoh.preprocess_ver as _ppv
#from AIiRPS.utils.dir_util import getResultFN
import GCoh.datconfig as datconf



import sys


def autocorrelate(signal, maxlag):
     AC = _N.empty(maxlag*2+1)
     sigN= signal.shape[0]
     lag0mag = _N.dot(signal, signal) / (sigN*sigN)
     for lg in range(1, maxlag+1):
          datlen = sigN-lg
          AC[maxlag-lg] = (_N.dot(signal[0:datlen], signal[lg:lg+datlen])/(datlen*datlen)) / lag0mag
          AC[maxlag+lg] = AC[maxlag-lg]
     AC[maxlag] = 1
     return AC

dataset =    datconf._RPS
c    = 2
if dataset == datconf._SIM:
    Fs   = 200
if dataset == datconf._RPS or dataset == datconf._STROOP:
    Fs   = 300
elif dataset == datconf._GONOGO:
    Fs   = 500
elif dataset == datconf._AAFFECT:
    Fs   = 500

ch_w_CM, rm_chs, list_ch_names, ch_types = datconf.getConfig(dataset, sim_nchs=10)
arr_ch_names = _N.array(list_ch_names)

ev_n   = 0


_WIDE = 0
_MED  = 1
_FINE = 2
_FINE1 = 3   #



#bin     = 512
#slide   = 64
_WIDE = 0
_FINE = 1

manual_cluster=False
armv_ver = 1
gcoh_ver = 3   #  bandwidth 7 ver 1, bandwidth 5 ver 2, bandwidth 9 ver 3

win, slideby      = _ppv.get_win_slideby(gcoh_ver)

hlfOverlap = int((win/slideby)*0.5)

#s = "../Neurable/DSi_dat/%(dsf)s_artfctrmvd_v%(av)d/%(dsf)s_gcoh_%(wn)d_%(sld)d_v%(av)d%(gv)d.dmp" % {"gf" : rpsm[dat], "dsf" : dat, "av" : armv_ver, "gv" : gcoh_ver, "wn" : bin, "sld" : slide}
#print("!!!!!!!!!!   %s" % s)
#lm         = depickle("../DSi_dat/%(dsf)s_artfctrmvd/v%(av)d/%(dsf)s_gcoh_%(wn)d_%(sld)d_v%(av)d%(gv)d.dmp" % {"gf" : rpsms.rpsm_eeg_as_key[dat], "dsf" : dat, "av" : armv_ver, "gv" : gcoh_ver, "wn" : win, "sld" : slideby})
lm         = depickle(datconf.getDataFN(dataset, "%(dsf)s_artfctrmvd/v%(av)d/%(dsf)s_gcoh_%(wn)d_%(sld)d_v%(av)d%(gv)d.dmp" % {"dsf" : dat, "av" : armv_ver, "gv" : gcoh_ver, "wn" : win, "sld" : slideby}))
# #lm         = depickle("../Neurable/DSi_dat/%(dat)s_gcoh_%(w)s_%(s)s.dmp" % {"dat" : dat, "w" : bin, "s" : slide})
# #A_gcoh_mat = _scio.loadmat("DSi_dat/%(dat)s_gcoh_%(w)d_%(sl)d.mat" % {"dat" : dat, "w" : bin, "sl" : slide})
# #A_gcoh     = A_gcoh_mat["Cs"]

strt       = 0  #  if start at middle of experiment
A_gcoh     = lm["Cs"][strt:]
n_fs       = lm["fs"]


outdir1     = datconf.getResultFN(dataset, "%(dir)s" % {"dir" : dat})
if not os.access(outdir1, os.F_OK):
     os.mkdir(outdir1)
outdir     = datconf.getResultFN(dataset, "%(dir)s/v%(av)d%(gv)d" % {"dir" : dat, "av" : armv_ver, "gv" : gcoh_ver})
if not os.access(outdir, os.F_OK):
     os.mkdir(outdir)
#  12.10.2020
################  egenvectors
#imag_evs  = A_gcoh_mat["VEC"][0]


imag_evs  = lm["VEC"][strt:, :, ev_n]

L_gcoh  = A_gcoh.shape[0]
nChs    = imag_evs.shape[2]
real_evs  = _N.empty((L_gcoh, n_fs.shape[0], nChs))

chs = lm["chs_picks"]
ch_names = arr_ch_names[chs].tolist()

for ti in range(L_gcoh):
    real_evs[ti] = _N.abs(imag_evs[ti])

mn = _N.mean(real_evs, axis=0)
sd = _N.std(real_evs, axis=0)

fs = lm["fs"]


#frngs = [[12, 18], [20, 25], [28, 35], [38, 45]]
#frngs = [[12, 18], [20, 25], [28, 35], [35, 42], [38, 45]]
#frngs = [[10, 15]]
#frngs = [[10, 18]]
#frngs = [[12, 18]]
#frngs = [[10, 15], [20, 25], [30, 40]]
#frngs = [[18, 25]]
frngs = [[15, 25]]
#frngs = [[35, 42]]

#frngs = [[25, 35]]
#frngs = [[43, 49]]
#frngs = [[40, 50]]
#frngs = [[30, 45]]
#frngs = [[38, 50]]
#frngs = [[32, 48]]
#frngs = [[35, 47]]
#frngs = [[8, 12], [12, 18]]
#frngs = [[33, 40], [34, 41], [35, 42], [36, 43], [37, 44]]
#frngs = [[12, 18], [18, 25], [25, 35], [35, 45]]
#frngs = [[22, 28], [35, 42]]
#frngs = [[28, 35], [38, 45]]

ignore_stored = True
pcs     = _N.empty(len(frngs))
minK    =11
maxK    =12
#minK = 6
#maxK = 7
try_Ks  = _N.arange(minK, maxK+1)
#TRs      = _N.array([1, 1, 3, 5, 10, 15, 20, 25, 25])  # more tries for higher K
TRs      = _N.array([1, 15, 20, 25, 25, 30, 40, 50, 60, 60, 60, 60, 60, 60, 80, 80, 80, 80])  # more tries for higher K
#TRs      = _N.array([60])  # more tries for higher K

bics = _N.ones(((maxK-minK), _N.max(TRs)))*1000000
labs = _N.empty((maxK-minK, _N.max(TRs), real_evs.shape[0]), dtype=_N.int)

nState_start = 0

for ich in range(len(frngs)):
    fL = frngs[ich][0]
    fH = frngs[ich][1]

    irngs = _N.where((fs > fL) & (fs < fH))[0]
    iL    = irngs[0]
    iH    = irngs[-1]    

    #Apr242020_16_53_03_gcoh_256_64
    nStates, rmpd_lab = find_or_retrieve_GMM_labels(dataset, dat, "%(gf)s_gcoh%(evn)d_%(wn)d_%(sld)d_v%(av)d%(gv)d" % {"gf" : dat, "av" : armv_ver, "gv" : gcoh_ver, "wn" : win, "sld" : slideby, "evn" : ev_n}, real_evs, iL, iH, fL, fH, armv_ver, gcoh_ver, which=0, try_K=try_Ks, TRs=TRs, ignore_stored=ignore_stored, manual_cluster=manual_cluster, do_pca=True, min_var_expld=0.99)
    ps = _N.arange(nStates)
    ps += nState_start
    nState_start += nStates

    #nStates, rmpd_lab = find_or_retrieve_GMM_labels(rpsm[dat], "%(dat)s_gcoh_%(w)d_%(s)d" % {"dat" : dat, "w" : bin, "s" : slide}, real_evs, iL, iH, fL, fH, which=0, try_K=try_Ks, TRs=TRs, log_transform=False)
    """
    ###############
    for K in range(minK, maxK):
        for tr in range(TRs[K]):
            gmm = mixture.GaussianMixture(n_components=K, covariance_type="full")

            gmm.fit(_N.sum(real_evs[:, iL:iH], axis=1))
            bics[K-minK, tr] = gmm.bic(_N.sum(real_evs[:, iL:iH], axis=1))
            labs[K-minK, tr] = gmm.predict(_N.sum(real_evs[:, iL:iH], axis=1))

    coords = _N.where(bics == _N.min(bics))
    print("min bic %.4e" % _N.min(bics))
    bestLab = labs[coords[0][0], coords[1][0]]   #  indices in 2-D array
    rmpd_lab = increasing_labels_mapping(bestLab)

    nStates =  list(range(minK, maxK))[coords[0][0]]
    """
    out_u = _N.mean(real_evs[:, iL:iH], axis=1)
    out = _N.empty((L_gcoh, nChs))
    iS  = 0
    for ns in range(nStates):
        ls = _N.where(rmpd_lab == ns)[0]
        out[iS:iS+len(ls)] = _N.mean(real_evs[ls, iL:iH], axis=1)
        iS += len(ls)

    iS = 0
    clrs  = ["black", "orange", "blue", "green", "red", "lightblue", "grey", "pink", "yellow", "brown", "cyan", "purple", "black", "orange", "blue", "green", "red", "black", "orange", "blue", "green", "red", "lightblue", "grey", "pink", "yellow", "brown", "cyan", "purple", "black", "orange", "blue", "green", "red"]
    W   = L_gcoh
    H   = nChs
    disp_wh_ratio = 3
    aspect = (W/H)/disp_wh_ratio
    unit = 2.5
    fig = _plt.figure(figsize=(disp_wh_ratio*unit + 1, 3*unit+unit/2))
    _plt.subplot2grid((2, 1), (0, 0))        
    _plt.title("1st GCoh eigenvector - temporal order")
    #fig.add_subplot(nStates+2, 1, 1)  
    _plt.imshow(out_u.T, aspect=aspect)
    _plt.ylim(-(nStates+2), nChs+0.1)
    for ns in range(nStates):
        nsx = _N.where(rmpd_lab == ns)[0]
        _plt.scatter(nsx, _N.ones(len(nsx))*ns - nStates - 1, color=clrs[ns], lw=1.5, s=4)
    _plt.xlim(0, L_gcoh)
    _plt.xlabel("(sample #) - not in experimental temporal order", fontsize=17)
    _plt.ylabel("electrode #", fontsize=16)
    _plt.xlabel("time bin", fontsize=16)
    _plt.xticks(fontsize=14)
    _plt.yticks(fontsize=14)

    _plt.subplot2grid((2, 1), (1, 0))        
    _plt.title("1st GCoh eigenvector - reordered by cluster label")
    #fig.add_subplot(nStates+2, 1, 1)    
    _plt.imshow(out.T, aspect=aspect)
    _plt.ylim(-(nStates+2), nChs+0.1)
    for ns in range(nStates):
        ls = _N.where(rmpd_lab == ns)[0]
        liS = iS
        iS += len(ls)
        _plt.plot([liS, iS], [ns-nStates-1, ns-nStates-1], color=clrs[ns], lw=3.5)
        if ns < nStates-1:
            _plt.axvline(x=iS, color="white", lw=1)
    _plt.xlim(0, L_gcoh)
    _plt.suptitle("%(ky)s   %(1)d-%(2)dHz    GCoh val: %(gcoh).3f   %(sts)s" % {"1" : fL, "2" : fH, "gcoh" : _N.mean(lm["Cs"][:, irngs]), "ky" : dat, "sts" : str(ps)})
    _plt.xlabel("(sample #) - not in experimental temporal order", fontsize=17)
    _plt.ylabel("electrode #", fontsize=16)
    _plt.xlabel("time bin", fontsize=16)
    _plt.xticks(fontsize=14)
    _plt.yticks(fontsize=14)

    iS = 0
    for ns in range(nStates):
        ls = _N.where(rmpd_lab == ns)[0]
        iS += len(ls)
        if ns < nStates-1:
            _plt.axvline(x=iS, color="white", lw=1)
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, hspace=0.3)

    _plt.savefig("%(od)s/%(dat)s_%(w)d_%(sl)d_clusters_coh_pattern_%(evn)d_%(1)d_%(2)d_v%(av)d%(gv)d" % {"1" : fL, "2" : fH, "dat" : dat, "w" : win, "sl" : slideby, "od" : outdir, "av" : armv_ver, "gv" : gcoh_ver, "evn" : ev_n}, transparent=True)
    #_plt.close()

    max_over_fs_each_state = _N.empty((nChs, nStates))
    for ns in range(nStates):
        ls = _N.where(rmpd_lab == ns)[0]
        mn_over_fs = _N.mean(real_evs[ls, iL:iH], axis=1)
        #min_all    = _N.min(mn_over_fs, axis=0)
        max_over_fs_each_state[:, ns]    = _N.max(mn_over_fs, axis=0)
    maxComp = _N.max(max_over_fs_each_state)

    all_vecs = _N.empty((nChs, nStates))
    
    for ns in range(nStates):
        ls = _N.where(rmpd_lab == ns)[0]
        mn_over_fs = _N.mean(real_evs[ls, iL:iH], axis=1)
        min_all    = _N.min(mn_over_fs, axis=0)
        all_vecs[:, ns] = (min_all / maxComp)*1e-5
    _sp.do_skull_plot_all_EVs(all_vecs, ps, ch_names, "%(od)s/%(dat)s_%(w)d_%(sl)d_skull_coh_pattern_%(evn)d_%(1)d_%(2)d_v%(av)d%(gv)d" % {"1" : fL, "2" : fH, "dat" : dat, "w" : win, "sl" : slideby, "av" : armv_ver, "gv" : gcoh_ver, "od" : outdir, "evn" : ev_n}, dat, fL, fH)
